# Summary Pemahaman TF-IDF

Jupyter ini merupakan translate dari artikel medium berikut:

Source: <br>
https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76
https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089 

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

Dalam notebook ini, kita akan melakukan simulasi sederhana TF-IDF menggunakan object dokumen sederhana:

In [2]:
documentA = 'the man went out for a walk'
documentB = 'the children sat around the fire'

## 1. Menghitung Bag of Words

Karena komputer tidak bisa memahami bahasa manusia secara langsung, maka bahasa manusia ini harus dikonversi menjadi vektor numerik. Salah satu teknik untuk mengambil informasi adalah bag of words (BoW):

In [3]:
bagOfWordsA = documentA.split(' ')
bagOfWordsB = documentB.split(' ')

In [4]:
bagOfWordsA

['the', 'man', 'went', 'out', 'for', 'a', 'walk']

In [5]:
bagOfWordsB

['the', 'children', 'sat', 'around', 'the', 'fire']

Untuk menyatukan informasi dokumen A dan B, kita gunakan fungsi `.union()`:

In [6]:
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB))
uniqueWords

{'a',
 'around',
 'children',
 'fire',
 'for',
 'man',
 'out',
 'sat',
 'the',
 'walk',
 'went'}

Pada dasarnya konsep Bag of Words mengkonversi informasi teks dengan cara menghitung jumlah kata pada satu dokumen object:

In [7]:
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1
numOfWordsB = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsB:
    numOfWordsB[word] += 1

In [32]:
bowA = pd.DataFrame(list(numOfWordsA.items()),columns = ['Item','bow_A'])
bowB = pd.DataFrame(list(numOfWordsB.items()),columns = ['Item','bow_B'])
print_dict = pd.merge(bowA, bowB, how="outer", on=["Item", "Item"])
print_dict

,Item,bow_A,bow_B
0,man,1,0
1,went,1,0
2,walk,1,0
3,for,1,0
4,a,1,0
5,around,0,1
6,children,0,1
7,out,1,0
8,sat,0,1
9,the,1,2


Dari kumpulan  kata-kata di atas, telihat masih ada komponen kata yang kurang memberikan arti semantik seperti `the` atau biasa disebut sebagai **stopwords**, sehingga kita dapat menghilangkan kata-kata kurang berarti ini menggunakan library `nltk`, namun untuk sementara tidak akan diaplikasikan disini:

In [14]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## 2. Menghitung Term of Frequency (TF)

TF dapat diartikan sebagai jumlah kemunculan suatu kata / jumlah total kata.

<img src="https://miro.medium.com/proxy/1*HM0Vcdrx2RApOyjp_ZeW_Q.png">

Sehingga jika suatu term `'t'` sering muncul pada suatu dokumen maka nilai `tf` akan tinggi dan merepresentasikan term tersebut cukup penting dalam dokumen tersebut.

In [33]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [16]:
tfA = computeTF(numOfWordsA, bagOfWordsA)
tfB = computeTF(numOfWordsB, bagOfWordsB)

In [39]:
tfA

{'man': 0.14285714285714285,
 'went': 0.14285714285714285,
 'walk': 0.14285714285714285,
 'for': 0.14285714285714285,
 'a': 0.14285714285714285,
 'around': 0.0,
 'children': 0.0,
 'out': 0.14285714285714285,
 'sat': 0.0,
 'the': 0.14285714285714285,
 'fire': 0.0}

## 3. Menghitung Inverse Document Frequency (IDF)

Document Frequency merupakan jumlah dokumen yang mengandung term `'t'`. IDF merupakan invers dari jumlah total dokumen dalam suatu korpus / DF.

<img src="https://miro.medium.com/proxy/1*A5YGwFpcTd0YTCdgoiHFUw.png">

Jika TF sudah mampu mengkonversi objek dokumen ke dalam bentuk vektor numerik, mengapa kita harus menghitung IDF juga?

Meskipun kata-kata yang tergolong stopwords akan dihilangkan sebelum divektorisasi, akan ada kata-kata yang memiliki kemunculan yang cenderung tinggi pada seluruh dokumen. 

Sehingga IDF dapat mengukur seberapa informatif term `'t'` pada suatu korpus:

- Untuk kata-kata yang tergolong sangat umum (misalnya stopwords), maka `df = N` sehingga `idf = log(1) = 0`
- Untuk kata-kata yang tergolong cukup banyak pada suatu dokumen namun tidak terlalu banyak dokumen yang mengandung term tersebut, maka `df <<` sehingga nilai `idf` akan tinggi


IDF juga memiliki kendala yakni jika pada suatu dokumen tidak terdapat kata `'w'` sehingga `df = 0` maka nilai `idf = log(N/0)` jadi bernilai tak hingga sehingga fungsi `idf` dismoothing dengan menambah angka 1 sebagai pembagi: `idf = log(N/(df+1))`


In [17]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

## 4. Menghitung TF-IDF

TF-IDF merupakan perkalian antara TF dan IDF.

<img src="https://miro.medium.com/proxy/1*A5YGwFpcTd0YTCdgoiHFUw.png">

Kategori term `'t'` dari suatu korpus berdasarkan nilai `TF`, `IDF` dan `TF-IDF` nya:

| Category | TF | IDF | TF-IDF | 
| :- | -: | -: | :-: |
| Common Words | High | Very Low | Very low
| Keyword of a document | High | High | Very high
| Unimportant words | Low | Low | Very low


In [18]:
idfs = computeIDF([numOfWordsA, numOfWordsB])

In [19]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [20]:
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
df = pd.DataFrame([tfidfA, tfidfB])

## TF-IDF menggunakan `TfidfVectorizer()`

Perhitungan TF-IDF sendiri sudah disediakan library-nya oleh sklearn:

In [22]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,around,children,fire,for,man,out,sat,the,walk,went
0,0.000000,0.000000,0.000000,0.42616,0.42616,0.42616,0.000000,0.303216,0.42616,0.42616
1,0.407401,0.407401,0.407401,0.00000,0.00000,0.00000,0.407401,0.579739,0.00000,0.00000
